In [2]:
import pandas as pd
import os, os.path as op
import numpy as np
from glob import glob
from functools import reduce

Wczytanie plików:
- Dane o kraju (Populacja, powierzchnia...)
- Dane o przewidywanej długości życia
- Dane o konsumpcji mięsa
- Dane o statystyce samobójstw

In [3]:
df = pd.read_csv(glob(op.join("data","population_by_country_2020*"))[0])
df_le = pd.read_csv(glob(op.join("data","PA_Life expectancy*"))[0])
df_mc = pd.read_csv(glob(op.join("data","PA_Meat consumption*"))[0])
df_sr = pd.read_csv(glob(op.join("data","PA_Suicide rate*"))[0])

- Usunięcie krajów ponizej 100_000 populacji\
- Usunięcie "%" z kolumn\
- Zastapienie braków danych średnią z kolumn

In [4]:
df = df[df["Population (2020)"] >= 100_000]
df["Urban Pop %"] = df["Urban Pop %"].apply(lambda x: x.replace(" %",""))
df["World Share"] = df["World Share"].apply(lambda x: x.replace(" %","")).astype(float)
df["Yearly Change"] = df["Yearly Change"].apply(lambda x: x.replace(" %","")).astype(float)
df['Med. Age'] = df['Med. Age'].astype(int)
df = df.replace("N.A.", np.nan)
df["Urban Pop %"] = df["Urban Pop %"].astype(float)
df["Urban Pop %"] = df["Urban Pop %"].fillna(df["Urban Pop %"].mean())
df.isna().any()


Country (or dependency)    False
Population (2020)          False
Yearly Change              False
Net Change                 False
Density (P/Km²)            False
Land Area (Km²)            False
Migrants (net)             False
Fert. Rate                 False
Med. Age                   False
Urban Pop %                False
World Share                False
dtype: bool

- Zmiana nazw kolumn
- Sortowanie alfabetyczne według nazw krajów
- Dodanie nowej kolumny

In [5]:
col_names = {'Country (or dependency)': 'Kraj',
 'Population (2020)': 'Populacja',
 'Yearly Change': 'Przyrost_rzeczywisty',
 'Net Change': 'Przyrost_rzeczywisty',
 'Density (P/Km²)': 'Zaludnienie',
 'Land Area (Km²)': 'Powierzchnia',
 'Migrants (net)': 'Migracje',
 'Fert. Rate': 'Wspolczynnik dzietności',
 'Med. Age': 'Sredni wiek',
 'Urban Pop %': 'Urbanizacja',
 'World Share': 'Powierzchnia ladu'}
df = df.rename(columns=col_names).sort_values("Kraj").reset_index(drop=True)


- Połączenie danych z trzech plików
- Usuniecie zbędnych kolumn
- Zastapienie braków danych średnią z kolumn

In [6]:
df_mc_le_sr = reduce(lambda x,y: pd.merge(x, y,on=['Country'], how='outer'), [df_mc, df_le, df_sr])
df_mc_le_sr = df_mc_le_sr.drop(["ISO-code_x", "ISO-code_y" , "ISO-code"], axis=1)
col_temp = ['Meat consumption', 'Life expectancy', 'Suicide rate']
df_mc_le_sr[col_temp] = df_mc_le_sr[col_temp].fillna(df_mc_le_sr[col_temp].mean())
df_mc_le_sr[col_temp].isna().sum()

Meat consumption    0
Life expectancy     0
Suicide rate        0
dtype: int64

Ilość pokrywajacych sie nazw krajów

In [7]:
df_mc_le_sr_kraj = [i for i in df_mc_le_sr["Country"]]
df_kraj = [i for i in df["Kraj"]]
common = list(set(df_mc_le_sr_kraj).intersection(df_kraj))
len(common)

177

Takie same kraje o innym zapisie

In [8]:
difference = []
for item in df_mc_le_sr_kraj:
    if item not in df_kraj:
        difference.append(item)
print(difference)


['American Samoa', 'Antigua and Barbuda', 'Cape Verde', 'Czech Republic', 'Democratic Republic of the Congo', 'Dominica', 'East Timor', 'FR Yugoslavia', 'Greenland', 'Ivory Coast', 'Republic of the Congo', 'Saint Kitts and Nevis', 'Saint Vincent and the Grenadines', 'Sao Tome and Principe', 'Seychelles', 'The Bahamas', 'The Gambia', 'Virgin Islands', 'Andorra', 'Liechtenstein', 'Marshall Islands', 'Palau', 'Palestine']


In [9]:
difference_2 = []
for item in df_kraj:
    if item not in df_mc_le_sr_kraj:
        difference_2.append(item)
print(difference_2)

['Aruba', 'Bahamas', 'Cabo Verde', 'Channel Islands', 'Congo', 'Curaçao', 'Czech Republic (Czechia)', "Côte d'Ivoire", 'DR Congo', 'Gambia', 'Guinea-Bissau', 'Macao', 'Mayotte', 'Puerto Rico', 'Réunion', 'Sao Tome & Principe', 'St. Vincent & Grenadines', 'State of Palestine', 'Taiwan', 'Timor-Leste', 'U.S. Virgin Islands', 'Western Sahara']


In [10]:
df_mc_le_sr['Country'] = df_mc_le_sr['Country'].replace([
    'The Gambia','The Bahamas','Sao Tome and Principe',
    'East Timor','Cape Verde','Czech Republic',
    'Democratic Republic of the Congo','Saint Vincent and the Grenadines',
    'Virgin Islands','Palestine'],
    ['Gambia','Bahamas','Sao Tome & Principe',
    'Timor-Leste','Cabo Verde','Czech Republic (Czechia)',
    'Congo','St. Vincent & Grenadines','U.S. Virgin Islands','State of Palestine'])

- Zmiana nazw kolumn
- Sortowanie alfabetyczne według nazw krajów
- Połączenie tabel

In [11]:
col_names_2 = {
    'Country': "Kraj", 'Meat consumption': "Spozycie miesa",
    'Life expectancy': "Przewidywana dlugosc zycia",
    'Suicide rate':"wspolczynnik samobojstw"
}
df_mc_le_sr = df_mc_le_sr.rename(columns=col_names_2).sort_values("Kraj").reset_index(drop=True)
df = df.merge(df_mc_le_sr, on='Kraj', how= 'inner')
df = df.round(2)
df.isna().any()


Kraj                          False
Populacja                     False
Przyrost_rzeczywisty          False
Przyrost_rzeczywisty          False
Zaludnienie                   False
Powierzchnia                  False
Migracje                      False
Wspolczynnik dzietności       False
Sredni wiek                   False
Urbanizacja                   False
Powierzchnia ladu             False
Spozycie miesa                False
Przewidywana dlugosc zycia    False
wspolczynnik samobojstw       False
dtype: bool

Dodanie nowej kolumny "Wielkosc", tak aby kraje były podzielone w prorocjach 50/50

In [12]:
df["Wielkosc"] = df["Populacja"].apply(lambda x:"Duzy" if x >= 9015361.0 else "Maly")


In [13]:
df.describe()

,Populacja,Przyrost_rzeczywisty,Przyrost_rzeczywisty,Zaludnienie,Powierzchnia,Migracje,Sredni wiek,Urbanizacja,Powierzchnia ladu,Spozycie miesa,Przewidywana dlugosc zycia,wspolczynnik samobojstw
count,1.940000e+02,194.000000,1.940000e+02,194.000000,1.940000e+02,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000
mean,3.995595e+07,1.261959,4.142524e+05,238.237113,6.621398e+05,80.376289,30.000000,58.731443,0.511701,43.477320,71.700773,9.673660
std,1.479640e+08,1.088556,1.216300e+06,816.294417,1.833044e+06,125278.800663,9.192247,21.957176,1.895947,34.211488,7.490160,5.226639
min,1.043980e+05,-1.350000,-3.838400e+05,2.000000,3.000000e+02,-653249.000000,15.000000,13.000000,0.000000,3.000000,52.800000,0.400000
25%,2.590332e+06,0.440000,5.323000e+03,32.500000,2.766750e+04,-10434.000000,21.250000,39.000000,0.030000,14.475000,66.625000,6.100000
50%,9.783438e+06,1.140000,8.181700e+04,82.000000,1.416550e+05,-1413.500000,29.500000,59.580000,0.125000,34.250000,72.350000,9.400000
75%,2.826828e+07,2.137500,3.615490e+05,194.000000,5.078675e+05,9488.250000,38.000000,77.500000,0.360000,67.400000,76.800000,11.775000
max,1.440298e+09,3.840000,1.358663e+07,8358.000000,1.637687e+07,954806.000000,48.000000,98.000000,18.470000,145.900000,84.700000,30.200000


#### Dodanie kolumny kontynent

In [14]:
df = df.drop(range(69,76,1))
df = df.reset_index(drop=True)
Europe = ["Albania","Austria","Belarus","Belgium","Bosnia and Herzegovina","Bulgaria"
          ,"Croatia","Cyprus","Czech Republic (Czechia)","Denmark","Estonia","Hungary","Iceland","","",
          "Finland","France","Germany","Greece","Ireland","Italy","Latvia",'Lithuania','Luxembourg','Malta'
         ,'Moldova','Montenegro','Netherlands','North Macedonia','Norway','Poland','Portugal','Romania'
         ,'Russia','Slovakia','Slovenia','Spain','Sweden','Switzerland','Ukraine','United Kingdom','Armenia']
America = ["Argentina","Bahamas","Barbados","Belize","Bolivia","Brazil","Canada"
           ,"Chile","Colombia","Costa Rica","Cuba","Dominican Republic","Ecuador","El Salvador",
           "French Guiana","Grenada","Guadeloupe","Guam","Guatemala","Guyana","Haiti","Honduras",
           "Jamaica","Kenya",'Martinique','Mexico','Nicaragua','Panama','Peru','Paraguay',"Saint Lucia",
          'St. Vincent & Grenadines','Suriname','Trinidad and Tobago','U.S. Virgin Islands','United States'
          ,'Uruguay','Venezuela']
Africa = ["Algeria","Angola","Benin","Botswana","Burkina Faso","Burundi","Cabo Verde",
          "Cameroon","Central African Republic","Chad","Comoros","Congo","Djibouti","Egypt","Equatorial Guinea","Eritrea"
          ,"Eswatini","Ethiopia","Gabon","Gambia","Guinea","Ghana",'Lesotho','Liberia','Libya'
          ,'Madagascar','Malawi','Mali','Mauritania','Mauritius','Morocco','Mozambique','Namibia','Niger'
         ,'Nigeria',"Rwanda",'Sao Tome & Principe','Senegal','Sierra Leone','Somalia','South Africa',
         'South Sudan','Sudan','Tanzania','Togo','Tunisia','Uganda','Zambia','Zimbabwe']
Asia = ["Afghanistan","Azerbaijan","Bahrain","Bangladesh","Bhutan","Brunei","Cambodia"
        ,"China","Georgia","Hong Kong","India","Indonesia","Iran","Iraq","Israel","Japan","Jordan",
        "Kazakhstan","Kuwait","Lebanon",'Kyrgyzstan','Laos','Malaysia','Maldives','Mongolia','Myanmar',
       'Nepal','North Korea','Oman','Pakistan','Philippines','Qatar','Saudi Arabia','Serbia','Singapore',
        'South Korea','Sri Lanka','State of Palestine','Syria','Tajikistan','Thailand','Timor-Leste',
        'Turkey','Turkmenistan','United Arab Emirates','Uzbekistan','Vietnam','Yemen'
       ]
Australia_oceania = ["Australia","Fiji","French Polynesia","Kiribati",'Micronesia','New Caledonia',
                     'New Zealand','Papua New Guinea',"Samoa",'Solomon Islands','Tonga','Vanuatu']
count_countines = {"Europe" : len(Europe), "America": len(America),
                   "Africa": len(Africa), "Asia": len(Asia), "Australia_oceania" :len(Australia_oceania)}
count_countines

{'Europe': 42,
 'America': 38,
 'Africa': 49,
 'Asia': 48,
 'Australia_oceania': 12}

In [15]:
difference = []
for item in df["Kraj"]:
    if item not in Europe and item not in America and item not in Africa and item not in Asia and item not in Australia_oceania:
        difference.append(item)
print(difference)

[]


In [16]:
regions = Europe + America + Africa + Asia + Australia_oceania
difference = [item for item in df["Kraj"] if item not in regions]
print(difference)

[]


In [17]:
continent_map = {"Asia": Asia, "America": America, "Africa": Africa, "Europe": Europe}

def add_continents(x):
    for continent, countries in continent_map.items():
        if x in countries:
            return continent
    return "brak danych"

In [18]:
df["Kontynent"] = df["Kraj"].apply(add_continents)
df = df.drop(163)
df = df.reset_index(drop=True)
df.head()

,Kraj,Populacja,Przyrost_rzeczywisty,Przyrost_rzeczywisty,Zaludnienie,Powierzchnia,Migracje,Wspolczynnik dzietności,Sredni wiek,Urbanizacja,Powierzchnia ladu,Spozycie miesa,Przewidywana dlugosc zycia,wspolczynnik samobojstw,Wielkosc,Kontynent
0,Afghanistan,39074280,2.33,886592,60,652860,-62920.0,4.6,18,25.0,0.50,17.30,64.50,6.4,Duzy,Asia
1,Albania,2877239,-0.11,-3120,105,27400,-14000.0,1.6,36,63.0,0.04,43.84,71.75,5.6,Maly,Europe
2,Algeria,43984569,1.85,797990,18,2381740,-10000.0,3.1,29,73.0,0.56,18.30,76.70,3.3,Duzy,Africa
3,Angola,33032075,3.27,1040977,26,1246700,6413.0,5.6,17,67.0,0.42,19.00,60.80,8.9,Duzy,Africa
4,Argentina,45267449,0.93,415097,17,2736690,4800.0,2.3,32,93.0,0.58,79.70,76.50,9.1,Duzy,America


In [19]:
#df.to_csv("Dane_Statystyka_Pełne.csv", index = False)